In [32]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("HUMETRO_LOCAL_RAG_EVAL_TEST")

LangSmith 추적을 시작합니다.
[프로젝트명]
HUMETRO_LOCAL_RAG_EVAL_TEST


In [34]:
import subprocess
from typing import Any
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma


# 2. 임베딩 모델 생성
def create_embeddings(model_name: str = "text-embedding-3-small") -> OpenAIEmbeddings:
    return OpenAIEmbeddings(model=model_name, dimensions=1536)


def load_vectorstore(persist_directory: str, embeddings: Any) -> Chroma:
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name="rag_documents",
    )
    print(f"벡터스토어 로드 완료. 문서 수: {vectorstore._collection.count()}")
    return vectorstore


# 3. 검색기(retriever) 생성 함수
def create_retriever(vectorstore: Chroma, k: int = 4) -> Any:
    return vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": k})


# 4. LLM 모델 생성 함수
def create_llm_ollama(model_name: str, temperature: float = 0.1) -> Any:
    if "ollama" in model_name:
        # Ollama 모델 확인 및 없으면 다운로드
        try:
            result = subprocess.run(
                ["ollama", "ps"], capture_output=True, text=True, check=True
            )
            running_servers = result.stdout.strip()

            # 2. 실행 중인 모델이 있으면 종료
            if running_servers:
                print("현재 실행 중인 모델을 종료합니다...")
                for line in running_servers.split("\n"):
                    if line and not line.startswith("NAME"):  # 헤더 행 제외
                        running_model = line.split()[0]
                        # 실행 중인 모델이 요청된 모델과 다른 경우에만 종료
                        if running_model != model_name:
                            print(f"모델 {running_model}을 종료합니다...")
                            subprocess.run(
                                ["ollama", "stop", running_model],
                                check=True,
                                capture_output=True,
                            )
            # 사용 가능한 모델 목록 확인
            result = subprocess.run(
                ["ollama", "list"], capture_output=True, text=True, check=True
            )
            available_models = result.stdout.lower()

            # 모델 이름에서 태그 분리
            if ":" in model_name:
                base_model = model_name.split(":")[0]
            else:
                base_model = model_name

            # 모델이 없으면 다운로드
            if base_model not in available_models:
                print(f"모델 {model_name}을 다운로드합니다...")
                subprocess.run(["ollama", "pull", model_name], check=True)
                print(f"모델 {model_name} 다운로드 완료")
            else:
                print(f"모델 {model_name}이 이미 존재합니다.")

        except subprocess.CalledProcessError as e:
            print(f"Ollama 명령 실행 중 오류 발생: {e}")
            print("Ollama가 설치되어 있고 실행 중인지 확인하세요.")
        except Exception as e:
            print(f"Ollama 모델 준비 중 오류 발생: {e}")

        # Ollama 모델 연결
        return Ollama(model=model_name, temperature=temperature)
    else:
        # OpenAI 모델 사용
        return ChatOpenAI(model=model_name, temperature=temperature)


def create_llm_lms(model_signature: str, temperature: float = 0.1) -> Any:
    if "gpt" in model_signature:
        return ChatOpenAI(model=model_signature, temperature=temperature)
    try:
        subprocess.run(["lms", "unload", "-a"])
        print("모든 lms 언로드 완료")
        subprocess.run(["lms", "load", model_signature])
        print(f"lms 모델 {model_signature} 로드 완료")
    except Exception as e:
        print(f"lms 명령 실행 중 오류 발생: {e}")
        print("lms가 설치되어 있고 실행 중인지 확인하세요.")
    return ChatOpenAI(base_url="http://localhost:1234/v1", model=model_signature)


# 5. RAG 체인 생성 함수
def create_rag_chain(llm: Any, retriever: Any) -> Any:
    # 한국의 역무환경을 고려한 RAG 프롬프트 템플릿
    template = """
당신은 한국의 도시철도 역무 지식 도우미입니다.
주어진 질문에 대해 제공된 문맥 정보를 기반으로 정확하고 도움이 되는 답변을 제공하세요.
문맥에 없는 내용은 답변하지 마세요. 모르는 경우 솔직히 모른다고 말하세요.

문맥 정보:
{context}

질문: {question}

답변:
"""

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([doc.page_content for doc in docs])

    # LCEL을 사용한 RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [ ]:
import json
import re
from tqdm import tqdm
from langchain_openai import OpenAIEmbeddings

embeddings = create_embeddings()
vectorstore = load_vectorstore(persist_directory="vectorstore", embeddings=embeddings)
if len(vectorstore.similarity_search("서울역 주변 명소")) == 0:
    raise ValueError("vectorstore is empty")
if len(vectorstore.get()["ids"]) == 0:
    raise ValueError("vectorstore is empty")

벡터스토어 로드 완료. 문서 수: 150


## 모델 정의

In [50]:
lms_models = {
    # "qwen3-1.7b": "qwen3-1.7b",
    "exaone-3.5-2.4b-instruct": "exaone-3.5-2.4b-instruct",
    # "kakaocorp.kanana-nano-2.1b-instruct": "kakaocorp.kanana-nano-2.1b-instruct",
    # "hyperclovax-seed-text-instruct-1.5b-hf-i1": "hyperclovax-seed-text-instruct-1.5b-hf-i1",
    # "qwen3-4b": "qwen3-4b",
    # "gpt-4o-mini": "gpt-4o-mini",
}
ollama_models = {
    "exaone": "exaone3.5",
    "clova": "yoonyoung/HyperCLOVAX-SEED-Vision-Instruct-3B",
    "kanana": "huihui_ai/kanana-nano-abliterated",  # 2.1b
    "llama": "benedict/linkbricks-llama3.1-korean:8b",  # 8b quantized
    "gpt-4o-mini": "gpt-4o-mini",
}


def sanitize_filename(filename):
    return re.sub(r"[^a-zA-Z0-9_.]", "_", filename)[:20] + ".json"


### 질문 데이터 불러오기

In [49]:
import pandas as pd

# 질문 데이터 불러오기
question_data = pd.read_csv("./translated_output.csv")
questions = list(question_data["user_input"])
print(questions[:10])


['도시철도법시행령에 따른 운임조정 절차는 어떻게 진행되나요?', '부산지하철의 운임 조정과 관련하여 시·도지사의 역할과 운임조정위원회의 구성 및 기능에 대해 자세히 알려주세요.', '부산 지하철의 각 노선에 걸친 엘리베이터 현황은 어떻게 되며, 특히 분포와 운영 상태에 대해 알려주세요?', '서면역에 수유실이 있나요?', '안녕하세요, 벡스코가 부산 지하철 시스템에서 에스컬레이터 유지보수를 위한 전용 장소 중 하나인지 알려주실 수 있나요?', '부산 도시철도 역세권 주차장 중 부산교통공사가 관리하는 주차 공간에 대한 상세 정보를 알려주세요. 주차 면수와 연락처를 포함해 주세요.', '안녕하세요, 부산시설공단이 지하철에서 관리하는 장소는 어디인가요?', '현재 구역 분류에 따른 수영 지역의 부산시 주차 요금', '동매역 1호선에서 지정된 만남의 장소 수에 대한 정보를 제공해 주실 수 있나요?', '안녕하세요, 수영역에 2호선과 3호선의 만남의 장소가 각각 몇 개 있는지 알려주실 수 있나요? 숫자와 관련된 것들이 좀 헷갈려서 부산에서 만남을 계획하는 데 필요하거든요. 이 정보로 도와주시면 감사하겠습니다.']


In [ ]:
for model_name, model_signature in lms_models.items():
    print(f"Loading {model_name}...")
    llm = create_llm_lms(model_signature)
    retriever = create_retriever(vectorstore)
    rag_chain = create_rag_chain(llm, retriever)
    filename = sanitize_filename(f"result_{model_name}.json")
    try:
        with open(filename, "r") as f:
            result_list = json.load(f)
    except FileNotFoundError:
        result_list = []

    # 이미 처리된 질문 제외
    processed_question = [i["question"] for i in result_list]
    questions = [i for i in questions if i not in processed_question]

    for question in tqdm(questions, desc=f"Evaluating {model_name}"):
        result = rag_chain.invoke(question)
        result_list.append({"question": question, "answer": result})
        if len(result_list) % 10 == 0:  # 10개마다 저장하기
            print(f"checkpoint, saved {len(result_list)}")
            with open(filename, "w") as f:
                json.dump(result_list, f, ensure_ascii=False)
    with open(filename, "w") as f:  # 최종 결과 저장하기
        json.dump(result_list, f, ensure_ascii=False)
    print()

# LANG SMITH CLIENT

In [38]:
# 새로운 질문 목록

from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET_DEMO"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset


dataset = create_dataset(client, dataset_name)

new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

# UI에서 업데이트된 버전 확인
client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)

{'example_ids': ['47e962c4-5edb-4455-9c9f-7c4cf2ba0f62',
  'ecc4ed95-7af1-470c-ab99-b58a1f1c1957'],
 'count': 2}